# SQL. Анализ базы данных сервиса для чтения книг.

***Задача:*** проанализировать базу данных о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В
свободное время жители городов больше не выходят на улицу, не посещают кафе
и торговые центры. Зато стало больше времени для книг. Это заметили
стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг
по подписке. Ваша первая задача как аналитика — проанализировать базу
данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские
обзоры книг. Эти данные помогут сформулировать ценностное предложение для
нового продукта.

***Задания***

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
    'pwd': '', # пароль
    'host': '',
    'port': , # порт подключения
    'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Описание данных

### Таблица books

Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

In [3]:
query = '''SELECT * FROM books'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


### Таблица authors 

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

In [4]:
query = '''SELECT * FROM authors'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


### Таблица publishers

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

In [5]:
query = '''SELECT * FROM publishers'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


### Таблица ratings

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

In [6]:
query = '''SELECT * FROM ratings'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


### Таблица reviews 

Содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

In [7]:
query = '''SELECT * FROM reviews'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


# Решение заданий.

## Сколько книг вышло после 1 января 2000 года

In [49]:
query = '''
    SELECT COUNT(book_id) as cnt_books
    FROM books
    WHERE publication_date::date > '01.01.2000'
    '''
pd.io.sql.read_sql(query, con = engine)

,cnt_books
0,819


В таблице books содержится информация о 819 книгах вышедших после 1 января 2000 года.

## Для каждой книги посчитать количество обзоров и среднюю оценку

In [50]:
query = '''
    SELECT b.book_id, 
        b.title,
        COALESCE(cnt_review, 0) as cnt_review,
        avg_rating 
    FROM books as b
    LEFT JOIN (SELECT book_id, COUNT(review_id) as cnt_review
               FROM reviews
               GROUP BY book_id) as rv ON rv.book_id = b.book_id
    LEFT JOIN (SELECT book_id, AVG(rating) as avg_rating
               FROM ratings
               GROUP BY book_id) as rt ON rt.book_id = b.book_id
    ORDER BY cnt_review DESC
    '''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,cnt_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,696,The Da Vinci Code (Robert Langdon #2),6,3.830508
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
996,808,The Natural Way to Draw,0,3.000000
997,221,Essential Tales and Poems,0,4.000000
998,387,Leonardo's Notebooks,0,4.000000


Пользователи оценили все книги от 1.5 до 5 баллов, а вот обзор написан не для всех. Шесть книг не получили пользовательских обзоров.

## Определить издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [52]:
query = '''
    SELECT p.publisher_id,
           publisher, 
           cnt_books
    FROM publishers as p
    JOIN (SELECT publisher_id,
          COUNT(book_id) as cnt_books
          FROM books
          WHERE num_pages > 50
          GROUP BY publisher_id) as b ON p.publisher_id = b.publisher_id
    ORDER BY cnt_books DESC
    LIMIT 1
    '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,cnt_books
0,212,Penguin Books,42


Издательство Penguin Books выпустило больше всего книг (толще 50 страниц) - 42шт

## Определить автора с самой высокой средней оценкой книг — учитывайть только книги с 50 и более оценками

In [103]:
query = '''
    SELECT a.author,
           ROUND(AVG(rt.rating),2) as avg_rating
    FROM authors as a
    JOIN books as b ON a.author_id = b.author_id
    JOIN (SELECT *,
          COUNT(rating_id) OVER (PARTITION BY book_id) as cnt_rating
          FROM ratings) as rt ON b.book_id = rt.book_id
    WHERE rt.cnt_rating >= 50
    GROUP BY a.author_id
    ORDER BY avg_rating DESC
    LIMIT 1
    '''
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Среди книг с 50 и более оценками самая высокая средняя оценка всех книг 4.29 у Джоан Роулинг(J.K. Rowling/Mary GrandPré)

## Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [130]:
query = '''
        SELECT ROUND(AVG(u.cnt_review), 2)
        FROM (SELECT rv.username, COUNT(review_id) as cnt_review
              FROM reviews as rv
              JOIN (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating) > 50) as rt ON rv.username = rt.username
              GROUP BY rv.username) as u
    '''
pd.io.sql.read_sql(query, con = engine)

,round
0,24.33


Самые активные пользователи, поставившие больше 50 оценк, в среднем пишут по 24 обзора.

# Выводы

Проанализировав предоставленную базу данных можно сказать, что:

- 819 книг вышло после 1 января 2000 года
- все книги имеют оценки пользователей (от 1.5 до 5 баллов), а вот обзор написан не для всех. Шесть книг не получили пользовательских обзоров.
- издательство Penguin Books выпустило больше всего книг (толще 50 страниц) - 42шт.
- среди книг с 50 и более оценками самая высокая средняя оценка всех книг 4.29 у Джоан Роулинг(J.K. Rowling/Mary GrandPré)
- самые активные пользователи, поставившие больше 50 оценк, в среднем пишут по 24 обзора.